In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [6]:
df = pd.read_csv('/content/Cleaned_data_to_run_ML.csv')

In [8]:
df = pd.DataFrame(df)

# Ensure date is in datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')  # Specifying format to avoid warning

# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Get unique products
products = df['Product Name'].unique()

# Store results
lstm_results = {}

# For each product
for product in products:
    print(f"Training LSTM model for {product}...")

    # Filter data for current product
    product_df = df[df['Product Name'] == product].sort_values('Date')

    # Check if we have enough data
    if len(product_df) < 50:  # Arbitrary minimum threshold
        print(f"Not enough data for {product}. Skipping.")
        continue

    # Use only the quantity for LSTM
    quantity_data = product_df['Total Quantity'].values.reshape(-1, 1)

    # Scale the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(quantity_data)

    # Create sequences
    seq_length = 30  # Using 30 days of history
    X, y = create_sequences(scaled_data, seq_length)

    # Skip if not enough sequences
    if len(X) < 10:
        print(f"Not enough sequences for {product} after creating windows. Skipping.")
        continue

    # Split into train and test
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Build LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Early stopping to prevent overfitting
    early_stop = EarlyStopping(monitor='val_loss', patience=10)

    # Train the model
    model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=1
    )

    # Evaluate
    y_pred = model.predict(X_test)

    # Inverse transform to get actual values
    y_test_inv = scaler.inverse_transform(y_test)
    y_pred_inv = scaler.inverse_transform(y_pred)

    # Calculate RMSE
    lstm_rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    lstm_results[product] = lstm_rmse
    print(f"Product: {product}, LSTM RMSE: {lstm_rmse:.2f}")

# Print summary of results
print("\nLSTM Results Summary:")
for product, rmse in lstm_results.items():
    print(f"Product: {product}, LSTM RMSE: {rmse:.2f}")

Training LSTM model for beef...
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - loss: 0.0604 - val_loss: 0.0197
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0218 - val_loss: 0.0201
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0231 - val_loss: 0.0198
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0228 - val_loss: 0.0216
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0231 - val_loss: 0.0195
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 0.0224 - val_loss: 0.0205
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0237 - val_loss: 0.0199
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0242 - val_loss: 0.0197
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0226 - val_loss: 0.0195
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0220 - val_loss: 0.0215
Epoch 11/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0231 - val_loss: 0.0197
Epoch 12/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - loss: 0.0957 - val_loss: 0.0257
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0256 - val_loss: 0.0241
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0236 - val_loss: 0.0254
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0239 - val_loss: 0.0256
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0234 - val_loss: 0.0211
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0247 - val_loss: 0.0212
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0240 - val_loss: 0.0213
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0231 - val_loss: 0.0211
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - loss: 0.0247 - val_loss: 0.0218
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0226 - val_loss: 0.0211
Epoch 11/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0250 - val_loss: 0.0217
Epoch 12/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0

...

44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 0.1284 - val_loss: 0.0234
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.0240 - val_loss: 0.0220
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0248 - val_loss: 0.0223
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0237 - val_loss: 0.0225
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0245 - val_loss: 0.0232
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.0252 - val_loss: 0.0230
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0242 - val_loss: 0.0219
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0236 - val_loss: 0.0243
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0231 - val_loss: 0.0254
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0232 - val_loss: 0.0272
Epoch 11/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0242 - val_loss: 0.0239
Epoch 12/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.0670 - val_loss: 0.0175
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0171 - val_loss: 0.0165
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0162 - val_loss: 0.0166
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0160 - val_loss: 0.0198
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0174 - val_loss: 0.0182
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0164 - val_loss: 0.0166
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0165 - val_loss: 0.0167
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0162 - val_loss: 0.0163
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0158 - val_loss: 0.0162
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0152 - val_loss: 0.0163
Epoch 11/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0157 - val_loss: 0.0162
Epoch 12/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - loss: 0.0693 - val_loss: 0.0218
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0233 - val_loss: 0.0170
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0235 - val_loss: 0.0239
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0241 - val_loss: 0.0201
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0224 - val_loss: 0.0186
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0235 - val_loss: 0.0198
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.0218 - val_loss: 0.0188
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0229 - val_loss: 0.0180
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 0.0216 - val_loss: 0.0183
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0198 - val_loss: 0.0171
Epoch 11/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.0208 - val_loss: 0.0183
Epoch 12/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Create DataFrame from the sample data
df = pd.DataFrame(df)

# Ensure date is in datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')  # Specifying format to avoid warning

# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Get unique products
products = df['Product Name'].unique()

# Store results
lstm_results = {}

# For each product
for product in products:
    print(f"Training LSTM model for {product}...")

    # Filter data for current product
    product_df = df[df['Product Name'] == product].sort_values('Date')

    # Check if we have enough data
    if len(product_df) < 50:  # Arbitrary minimum threshold
        print(f"Not enough data for {product}. Skipping.")
        continue

    # Use only the quantity for LSTM
    quantity_data = product_df['Total Quantity'].values.reshape(-1, 1)

    # Scale the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(quantity_data)

    # Create sequences
    seq_length = 30  # Using 30 days of history
    X, y = create_sequences(scaled_data, seq_length)

    # Skip if not enough sequences
    if len(X) < 10:
        print(f"Not enough sequences for {product} after creating windows. Skipping.")
        continue

    # Split into train and test
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Build LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Early stopping to prevent overfitting
    early_stop = EarlyStopping(monitor='val_loss', patience=10)

    # Train the model with 50 epochs instead of 100
    model.fit(
        X_train, y_train,
        epochs=50,  # Changed from 100 to 50
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=1
    )

    # Evaluate
    y_pred = model.predict(X_test)

    # Inverse transform to get actual values
    y_test_inv = scaler.inverse_transform(y_test)
    y_pred_inv = scaler.inverse_transform(y_pred)

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)

    # Store results
    lstm_results[product] = {
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    }

    print(f"Product: {product}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAE: {mae:.2f}")
    print(f"  R²: {r2:.4f}")

# Print summary of results
print("\nLSTM Results Summary:")
for product, metrics in lstm_results.items():
    print(f"Product: {product}")
    print(f"  RMSE: {metrics['RMSE']:.2f}")
    print(f"  MAE: {metrics['MAE']:.2f}")
    print(f"  R²: {metrics['R²']:.4f}")

Training LSTM model for beef...
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 0.1020 - val_loss: 0.0198
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0227 - val_loss: 0.0195
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0241 - val_loss: 0.0194
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0227 - val_loss: 0.0194
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0221 - val_loss: 0.0198
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0239 - val_loss: 0.0211
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0238 - val_loss: 0.0198
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0213 - val_loss: 0.0194
Epoch 9/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0221 - val_loss: 0.0199
Epoch 10/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0222 - val_loss: 0.0197
Epoch 11/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 0.0220 - val_loss: 0.0201
Epoch 12/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0229 - val_l

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.0752 - val_loss: 0.0250
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0240 - val_loss: 0.0218
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0240 - val_loss: 0.0221
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0230 - val_loss: 0.0212
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0239 - val_loss: 0.0222
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0228 - val_loss: 0.0220
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0243 - val_loss: 0.0213
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0230 - val_loss: 0.0238
Epoch 9/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0236 - val_loss: 0.0256
Epoch 10/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0227 - val_loss: 0.0218
Epoch 11/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 0.0227 - val_loss: 0.0222
Epoch 12/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0231 - val_l

...

44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - loss: 0.0722 - val_loss: 0.0266
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0266 - val_loss: 0.0225
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0225 - val_loss: 0.0219
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0235 - val_loss: 0.0228
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0231 - val_loss: 0.0249
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0240 - val_loss: 0.0220
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.0260 - val_loss: 0.0257
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0248 - val_loss: 0.0220
Epoch 9/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0244 - val_loss: 0.0231
Epoch 10/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0235 - val_loss: 0.0233
Epoch 11/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0244 - val_loss: 0.0233
Epoch 12/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 0.0218 - val_l

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - loss: 0.0731 - val_loss: 0.0185
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0175 - val_loss: 0.0187
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0174 - val_loss: 0.0165
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0178 - val_loss: 0.0163
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0169 - val_loss: 0.0166
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0169 - val_loss: 0.0162
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 0.0163 - val_loss: 0.0166
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0172 - val_loss: 0.0169
Epoch 9/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.0158 - val_loss: 0.0176
Epoch 10/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0163 - val_loss: 0.0167
Epoch 11/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.0166 - val_loss: 0.0176
Epoch 12/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 0.0169 - val_l

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.0433 - val_loss: 0.0170
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0229 - val_loss: 0.0181
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 0.0218 - val_loss: 0.0180
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0229 - val_loss: 0.0224
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0243 - val_loss: 0.0229
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0249 - val_loss: 0.0208
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0242 - val_loss: 0.0175
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0237 - val_loss: 0.0210
Epoch 9/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0225 - val_loss: 0.0175
Epoch 10/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 0.0220 - val_loss: 0.0200
Epoch 11/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0213 - val_loss: 0.0221
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
Product: soybeans
  RMSE: 8.87
  M